In [ ]:
# IO packages
from aicsimageio import AICSImage
import os
import imageio

# calculation packages
import numpy as np

# segmentation packages
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from aicssegmentation.core.seg_dot import dot_3d_wrapper
from skimage.morphology import dilation, ball, remove_small_objects

# watershed packages
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy.ndimage import distance_transform_edt
from skimage.measure import label



In [ ]:
def segment_centrosomes_interphase(image_name, filepath, s3_param, output_path):
    """ Takes a filepath that is a single channel tif as an input
    Segments the image - optimized for centrosomes
    Returns a segmented image
    """
        
    print('Segmenting centrosomes for ' + filepath)

    
    reader = AICSImage(filepath) 
    IMG = reader.data

    #####################
    structure_channel = 0
    #####################

    structure_img0 = IMG[0,structure_channel,:,:,:]
    
    ################################
    ## PARAMETER ##
    intensity_scaling_param = [0]
    gaussian_smoothing_sigma = 1
    ################################
    
    # intensity normalization
    structure_img = intensity_normalization(structure_img0, scaling_param=intensity_scaling_param)

    # smoothing with gaussian filter
    structure_img_smooth = image_smoothing_gaussian_slice_by_slice(structure_img, sigma=gaussian_smoothing_sigma)
    
    ################################
    ## Detect spots w/ 3D dot wrapper ##
    bw = dot_3d_wrapper(structure_img_smooth, s3_param)

    ################################
    ## PARAMETERS for removing small objects ##
    minArea = 100
    ################################

    final_seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)
    
    ################################
    ## SAVE TO DISK
    ################################
    
    output_seg = final_seg>0
    out=output_seg.astype(np.uint8)
    out[out>0]=255
    imageio.volwrite(output_path + image_name, out)

        
    return None

In [ ]:
def segment_centrosomes_metaphase(image_name, filepath, s3_param, output_path):
    """ Takes a filepath that is a single channel tif as an input
    Segments the image - optimized for centrosomes
    Returns a segmented image
    """
        
    print('Segmenting centrosomes for ' + filepath)

    
    reader = AICSImage(filepath) 
    IMG = reader.data

    #####################
    structure_channel = 0
    #####################

    structure_img0 = IMG[0,structure_channel,:,:,:]
        
    minimum_value = np.amin(structure_img0)
    mean_value = np.mean(structure_img0)
    percentile_99 = np.percentile(structure_img0, 99.99)
    std_array = np.std(structure_img0)

    a = round((mean_value - minimum_value) / std_array, 1)
    b = round((percentile_99 - mean_value) / std_array, 1)
    
    ################################
    ## PARAMETER ##
    intensity_scaling_param = [a, b]
    gaussian_smoothing_sigma = 1
    ################################
    
    # intensity normalization
    structure_img = intensity_normalization(structure_img0, scaling_param=intensity_scaling_param)

    # smoothing with gaussian filter
    structure_img_smooth = image_smoothing_gaussian_slice_by_slice(structure_img, sigma=gaussian_smoothing_sigma)
    
    ################################
    ## Detect spots w/ 3D dot wrapper ##
    ## Parameters are optimized to stage and RNA type ##
    bw = dot_3d_wrapper(structure_img_smooth, s3_param)

    ################################
    ## PARAMETERS for removing small objects ##
    minArea = 100
    ################################

    final_seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)
    
    ################################
    ## SAVE TO DISK
    ################################
    
    output_seg = final_seg>0
    out=output_seg.astype(np.uint8)
    out[out>0]=255
    imageio.volwrite(output_path + image_name, out)

        
    return None

In [ ]:
# segment interphase centrosomes

centrosome_raw_data_dir = '/Users/pearlryder/data/fmr-data-2/data/centrosomes/raw-data-by-stage/interphase'

output_path = '/Users/pearlryder/data/fmr-data-2/data/centrosomes/segmentations/'

image_ls = os.listdir(centrosome_raw_data_dir)

s3_param = [[1, 0.02]]

for image_name in image_ls:
    if not image_name[0] == '.':
        image_path = os.path.join(centrosome_raw_data_dir, image_name)

        segment_centrosomes_interphase(image_name, image_path, s3_param, output_path)


    
    


In [ ]:
# segment metaphase centrosomes using different parameters for NC 10 and NC 13

s3_params_dict = {'NC10': [[1, 0.03]],
                 'NC13': [[1, 0.06]]}


In [ ]:
# segment metaphase centrosomes using different parameters for NC 10 and NC 13

rna_dirs_to_process = ['NC10', 'NC13']

filepath_to_dirs = '/Users/pearlryder/data/fmr-data-2/data/centrosomes/raw-data-by-stage/metaphase/'

output_path = '/Users/pearlryder/data/fmr-data-2/data/centrosomes/segmentations/'

for rna_dir in rna_dirs_to_process:
    
    dir_path = os.path.join(filepath_to_dirs, rna_dir)
    
    image_ls = os.listdir(dir_path)
    
    for image_name in image_ls:
        if not image_name[0] == '.':
            image_path = os.path.join(dir_path, image_name)

            s3_param = s3_params_dict[rna_dir]

            segment_centrosomes_metaphase(image_name, image_path, s3_param, output_path)
        

    
    
